# Recommender System WBSFLIX

In [44]:
import pandas as pd
import numpy as np

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
#find paranthesis to extract the year from the title
#st[st.find("(")+1:st.rfind(")")]

Density of the matrix, amount of filled cells that are not zero - high density wanted for recommender systems - means lots of data available
Sparsity, 1 - density, amount of zeros in the matrix - sparse matrixes have a high amount of zeros

In [27]:
#ratings.shape (100836, 4), no ratings with 0, there are only rated films in the dataset
ratings.query('rating == 0').shape

(0, 4)

In [22]:
# many films have only few ratings, the last (fourth) quartile has ratings from 9 to 329 so a threshold for taking ratings into account will be set to >= 9
ratings.query('rating != 0').groupby('movieId').agg(count_rating=('rating','count')).sort_values(by='count_rating', ascending=False).describe()

,count_rating
count,9724.000000
mean,10.369807
std,22.401005
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,329.000000


In [40]:
agg_ratings = ratings.query('rating != 0').groupby('movieId').agg(count_rating=('rating','count'),mean_rating=('rating','mean'))

In [48]:
agg_ratings = agg_ratings.query('count_rating >= 9').sort_values(by=['mean_rating', 'count_rating'], ascending=False).assign(log_rating=agg_ratings['mean_rating']*np.log(agg_ratings['count_rating'])).head(50)#.describe()
agg_ratings.sort_values(by='log_rating', ascending=False).head(20)

,count_rating,mean_rating,log_rating
movieId,,,
318,317,4.429022,25.506303
2959,218,4.272936,23.007602
858,192,4.289062,22.549726
1221,129,4.259690,20.701294
1213,126,4.250000,20.554198
48516,107,4.252336,19.870440
912,100,4.240000,19.525922
750,97,4.268041,19.525055
904,84,4.261905,18.883719


# Popularity Rating

In [42]:
def popularity_rating(df,num):
    output = (
    df
        .query('rating != 0')
        .groupby('movieId')
        .agg(count_rating=('rating','count'),mean_rating=('rating','mean'))
    )
    output = output.query('count_rating >= 9').sort_values(by=['mean_rating', 'count_rating'], ascending=False)
    return output.head(num)

In [43]:
popularity_rating(ratings, 20)

,count_rating,mean_rating
movieId,,
1041,11,4.590909
3451,11,4.545455
1178,12,4.541667
1104,20,4.475000
2360,12,4.458333
1217,15,4.433333
318,317,4.429022
951,14,4.392857
5008,9,4.388889
